In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
df = pd.read_csv("../input/input_1.csv")
df

,date,sentiment,clasificacion,id
0,2021-08-01,NEUTRAL,Otro,0
1,2021-08-01,NEUTRAL,Otro,1
2,2021-08-01,NEUTRAL,Otro,2
3,2021-08-01,NEUTRAL,Otro,3
4,2021-08-01,NEUTRAL,APP,4
...,...,...,...,...
464970,2022-08-10,NEUTRAL,Otro,464970
464971,2022-08-10,POSITIVE,Otro,464971
464972,2022-08-10,NEUTRAL,Otro,464972
464973,2022-08-10,NEUTRAL,Otro,464973
